In [3]:
import numpy as np
from fenics import *

In [13]:
# Imports for plotting
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
# Has side effects allowing for 3D plots
from mpl_toolkits.mplot3d import Axes3D

In [27]:
### Mesh sizes
t, T = 0.0, 10.0   # start and end times
nt = 100            # number of time steps
a, b = 0, 40     # start and end spacial positions
nx = 100           # number of space steps
dt = (T - t)/ nt

### Define periodic boundary
class PeriodicBoundary(SubDomain):

    def inside(self, x, on_boundary):
        return bool(x[0] < DOLFIN_EPS and x[0] > -DOLFIN_EPS and on_boundary)

    # Map right boundary to left boundary
    def map(self, x, y):
        y[0] = x[0] - (b - a)

### Create mesh and define function spaces
mesh = IntervalMesh(nx, a, b)
F_ele = FiniteElement("CG", mesh.ufl_cell(), 1)
V = FunctionSpace(mesh, F_ele, constrained_domain=PeriodicBoundary())
W = FunctionSpace(mesh, MixedElement([F_ele, F_ele]), constrained_domain=PeriodicBoundary())

In [28]:
### Initial values
u0_uninterpolated = Expression('2/cosh(x[0] - 403.0/15.0) + 5 / cosh(x[0] - 203.0/15.0)', degree=2)
u0 = interpolate(u0_uninterpolated, V)

# Find initial value m0 for m
q = TestFunction(V)
m0 = TrialFunction(V)

am0 = q * m0 * dx
Lm0 = (q * u0 + q.dx(0) * u0.dx(0)) * dx
      
m0 = Function(V)
solve(am0 == Lm0, m0)

In [29]:
### Storing results
# We begin by storing the intial condition.
uvals = [u0.compute_vertex_values(mesh)]

In [38]:
### Define variational problem
# Stores the values of the previous time step. Starts off at the initial condition.
m_prev, u_prev = m0, u0
# Test functinos
p, q = TestFunctions(W)
# Function to solve for
w = Function(W)
m, u = w.split()
# m.assign(m0)
# u.assign(u0)
m_mid = 0.5 * (m + m_prev)
u_mid = 0.5 * (u + u_prev)
F = (
    (q * u + q.dx(0) * u.dx(0) - q * m) * dx +                                          # q part
    (p * (m - m_prev) + dt * (p * m_mid * u_mid.dx(0) - p.dx(0) * m_mid * u_mid)) * dx  # p part
    )

# L = lhs(F)
# J = derivative(F, w)
# problem = NonlinearVariationalProblem(F, w, [], J)
# solver = NonlinearVariationalSolver(problem)

# Time-stepping
for n in range(nt):
    # Update current time
    t += dt
    # Print energy
    E = assemble((u_prev * u_prev + u_prev.dx(0) * u_prev.dx(0)) * dx)
    print("time {:0>4} energy {}".format(t, E))
    # Compute solution
    solve(F == 0, w)
#     solver.solve()
    # Save result
    uvals.append(u.compute_vertex_values(mesh))
    # Update for next loop
    u_prev.assign(u)
    m_prev.assign(m)
print('done')

time 00.2 energy 76.47262639986486


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'MatSetValuesLocal'.
*** Reason:  PETSc error code is: 63 (Argument out of range).
*** Where:   This error was encountered inside /home/conda/feedstock_root/build_artifacts/fenics_1530480676164/work/dolfin/la/PETScMatrix.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2018.1.0
*** Git changeset:  6bc297d2abc96b1a4f9dd884fd1cdbd0dbe7c18e
*** -------------------------------------------------------------------------


In [ ]:
uvals = np.array(uvals)
xvals = mesh.coordinates()[:, 0]
tvals = np.linspace(0, T, nt + 1)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(*np.meshgrid(xvals, tvals), uvals)